#### input : osmnx 라이르러리
#### output : ./make_file/division/before/서울시_자동차도로_분할_{숫자}.csv

In [1]:
import osmnx as ox 
import numpy as np

In [2]:
# 광진구의 골목길 포함 도로 네트워크 가져오기 (residential, service, tertiary 도로 포함)
place_name = "서울, 대한민국"
graph = ox.graph_from_place(place_name, network_type='drive') 


In [3]:
# 모든 도로의 시작점과 종료점 노드 정보
nodes, edges = ox.graph_to_gdfs(graph) 


In [4]:
print(f'불러온 nodes : {len(nodes)}')
print(f'불러온 edges : {len(edges)}') 

불러온 nodes : 68194
불러온 edges : 189829


In [5]:
# edges 데이터프레임의 구조와 첫 몇 개의 행을 확인
print(edges.head())
print(edges.info())

# nodes 데이터프레임의 구조와 첫 몇 개의 행을 확인
print(nodes.head())
print(nodes.info())


                              osmid      highway    name  oneway reversed  \
u         v          key                                                    
266813237 3771909762 0    373776969  residential  보광로12길   False    False   
          436847094  0    373776969  residential  보광로12길   False     True   
          3771899749 0    373776966  residential  보광로14길   False    False   
278159482 1378780898 0     25524218  residential   보광로7길   False    False   
          1378780857 0     97836950     tertiary     장문로   False    False   

                             length  \
u         v          key              
266813237 3771909762 0    27.605333   
          436847094  0    20.002271   
          3771899749 0    69.877541   
278159482 1378780898 0    66.892380   
          1378780857 0    22.682960   

                                                                   geometry  \
u         v          key                                                      
266813237 3771909762 0    LI

In [6]:
try :
    from tqdm import tqdm
except :
    !pip install tqdm
    from tqdm import tqdm

In [7]:
# 삭제할 도로들의 리스트
delete_road_type = ['motorway', 'path', 'cycleway', 'road', 
                    'trunk', 'trunk_link',
                    'primary', 'primary_link',
                    'secondary', 'secondary_link',
                    'pedestrian', 'track']

In [8]:
# 전역 배열 초기화
road_names = []
highway_types = []
start_lats = []
start_lons = []
end_lats = []
end_lons = []
road_len = []
tunnel_status = []  
bridge_status = [] 
service_status = []   
road_ids = []
max_speeds = []
lane_counts = []
one_way = []
junction_status = []

# 도로 종류 한글 매핑 딕셔너리
road_type_map = {
    'motorway': '고속도로',
    'trunk': '간선도로',
    'primary': '주도로',
    'secondary': '2차로',
    'tertiary': '3차로',
    'unclassified': '비분류 도로',
    'residential': '주거지 도로',
    'service': '서비스 도로',
    'living_street': '생활도로'
}

# 'edges' 데이터프레임에서 각 도로에 대해 정보 추출
for index, row in tqdm(edges.iterrows(), total=len(edges)): 
    # MultiIndex에서 u와 v 값 추출
    start_node = index[0]  # 시작 노드 (u)
    end_node = index[1]    # 종료 노드 (v)

    # 노드 정보를 기반으로 위도와 경도 가져오기
    try:
        start_lat, start_lon = nodes.loc[start_node, ['y', 'x']]
        end_lat, end_lon = nodes.loc[end_node, ['y', 'x']]
    except KeyError:
        # 노드 정보가 없을 경우 건너뛰기
        continue

    # 도로명(속성 name)과 도로 특성 확인
    road_name = row.get('name', None)  # 도로명이 없는 경우 None 처리
    highway_type = row.get('highway', None)  
    # if highway_type not in ['unclassified', 'motorway', 
    #                         'trunk', 'trunk_link', 'primary', 'primary_link', 
    #                         'secondary', 'secondary_link', 'residential', 'tertiary', 
    #                         'living_street', 'road', 'pedestrian',
    #                         'path', 'service', 'cycleway'] :
    #     print(highway_type)                              
    tunnel_status_val = row.get('tunnel', None)   
    bridge_status_val = row.get('bridge', None)  # 교량 여부
    service_status_val = row.get('service', None)  # 서비스 도로 여부
    maxspeed = row.get('maxspeed', None)  # 속도 제한
    lanes = row.get('lanes', None)  # 차선 수
    oneway = row.get('oneway', None)  # 일방통행 여부
    junction = row.get('junction', None)  # 교차로 여부
    length = row.get('length', None)  
    
    if isinstance(highway_type, list): 
        continue 
        
    if not highway_type:  # None, 빈 문자열, 빈 리스트 등을 모두 포함
        continue

    if highway_type == 'unclassified' :
        continue
        
    # 규모가 큰 도로나, 등산로, 산책로 등은 제거
    if highway_type in delete_road_type :
        continue  # 해당 도로는 건너뛰기 
        
    # 전역 배열에 데이터 추가
    road_names.append(road_name if road_name else None)  # 값이 없으면 None로 처리
    # 도로 종류를 한글로 매핑
    road_type_in_korean = road_type_map.get(highway_type, None)
    highway_types.append(road_type_in_korean)  # 값이 없으면 None로 처리
    start_lats.append(start_lat if start_lat else None)  # 값이 없으면 None로 처리
    start_lons.append(start_lon if start_lon else None)  # 값이 없으면 None로 처리
    end_lats.append(end_lat if end_lat else None)  # 값이 없으면 None로 처리
    end_lons.append(end_lon if end_lon else None)  # 값이 없으면 None로 처리
    road_len.append(length if length else None)
    tunnel_status.append(tunnel_status_val if tunnel_status_val else None)  # 값이 없으면 None로 처리
    bridge_status.append(bridge_status_val if bridge_status_val else None)  # 값이 없으면 None로 처리
    service_status.append(service_status_val if service_status_val else None)  # 값이 없으면 None로 처리
    road_ids.append(row['osmid'] if row['osmid'] else None)  # 값이 없으면 None로 처리
    max_speeds.append(maxspeed if maxspeed else None)  # 값이 없으면 None로 처리
    lane_counts.append(lanes if lanes else None)  # 값이 없으면 None로 처리
    one_way.append(oneway if oneway else None)  # 값이 없으면 None로 처리
    junction_status.append(junction if junction else None)  # 값이 없으면 None로 처리


100%|█████████████████████████████████| 189829/189829 [00:30<00:00, 6313.76it/s]


  9%|███▍                                  | 658/7216 [00:00<00:00, 6576.06it/s]

 18%|██████▋                              | 1316/7216 [00:00<00:00, 6578.11it/s]

 28%|██████████▏                          | 1993/7216 [00:00<00:00, 6662.19it/s]

 37%|█████████████▋                       | 2660/7216 [00:00<00:00, 6606.69it/s]

...

 64%|███████████████████████▋             | 4626/7216 [00:00<00:00, 6449.81it/s]

 73%|███████████████████████████          | 5287/7216 [00:00<00:00, 6498.00it/s]

 83%|██████████████████████████████▌      | 5965/7216 [00:00<00:00, 6581.95it/s]

 92%|██████████████████████████████████   | 6638/7216 [00:01<00:00, 6624.64it/s]

100%|█████████████████████████████████████| 7216/7216 [00:01<00:00, 6572.27it/s]

In [9]:
# 각 배열의 길이 확인
print(len(road_names), len(highway_types))
print(len(start_lats), len(start_lons))
print(len(end_lats), len(end_lons))

print(f'{len(road_len)}')


161645 161645
161645 161645
161645 161645
161645


In [10]:
average_lat = []
average_lon = []

for i in range(max(len(start_lats), len(end_lats))):
    start_lat = start_lats[i] if i < len(start_lats) else start_lats[-1]
    end_lat = end_lats[i] if i < len(end_lats) else end_lats[-1]
    average_lat.append((start_lat + end_lat) / 2)

for i in range(max(len(start_lons), len(end_lons))):
    start_lon = start_lons[i] if i < len(start_lons) else start_lons[-1]
    end_lon = end_lons[i] if i < len(end_lons) else end_lons[-1]
    average_lon.append((start_lon + end_lon) / 2)


In [17]:
# 데이터프레임 생성
data = {
    '도로명': road_names,
    '도로 종류': highway_types,
    '시작점_위도': start_lats,
    '시작점_경도': start_lons,
    '종료점_위도': end_lats,
    '종료점_경도': end_lons,
    '중앙점_위도' : average_lat,
    '중앙점_경도' : average_lon,
    '도로_길이' : road_len
}


In [18]:
import pandas as pd

df = pd.DataFrame(data)


In [19]:
df.isnull().sum()

도로명       12745
도로 종류      1310
시작점_위도        0
시작점_경도        0
종료점_위도        0
종료점_경도        0
중앙점_위도        0
중앙점_경도        0
도로_길이         0
dtype: int64

In [20]:
df.columns

Index(['도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '도로_길이'],
      dtype='object')

In [24]:
len_df = len(df)
chunk_size = 10000  # 1만 개씩 나누기
num_chunks = np.ceil(len_df / chunk_size).astype(int)

In [34]:
df['도로명'] = df['도로명'].ffill()

In [35]:
df['도로명'].isnull().sum()

np.int64(0)

In [47]:
df['도로명'].value_counts()

도로명
통일로                    227
한천로                    227
통일로 중앙버스전용차로           217
남부순환로                  204
오패산로                   195
                      ... 
[월드컵북로, 월드컵로36길]         1
[도봉로114길, 노해로62길]        1
헌릉IC3교                   1
[새터산14길, 새터산16길]         1
[덕릉로131길, 덕릉로131가길]      1
Name: count, Length: 12905, dtype: int64

In [48]:
try:
    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, len_df)
        chunk = df.iloc[start_idx:end_idx]
        
        save_url = f'./make_file/division/before/서울시_자동차도로_분할_{i+1:02d}.csv'
        chunk.to_csv(save_url, encoding="UTF-8", index=False)
        print(f"{save_url} 데이터 저장 완료.")
except OSError as e:
    print(e)


./make_file/division/before/서울시_자동차도로_분할_01.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_02.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_03.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_04.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_05.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_06.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_07.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_08.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_09.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_10.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_11.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_12.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_13.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_14.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_15.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_16.csv 데이터 저장 완료.
./make_file/division/before/서울시_자동차도로_분할_17.csv 데이터 저장 완